In [26]:
#extract images.py
import glob
from shutil import copyfile

emotions = ["neutral", "anger", "contempt", "disgust", "fear", "happy", "sadness", "surprise"] #Define emotion order
participants = glob.glob("source_emotion\\*") #Returns a list of all folders with participant numbers
for x in participants:
    part = "%s" %x[-4:] #store current participant number
    for sessions in glob.glob("%s\\*" %x):#Store list of sessions for current participant
        for files in glob.glob("%s\\*" %sessions):
            #print(files)
            current_session = files[20:-30] #[20:-30]
            print(current_session)
            file = open(files, 'r')
            #print(file)
            emotion = int(float(file.readline())) #emotions are encoded as a float, readline as float, then convert to integer.
            #print(emotion)
            sourcefile_emotion = glob.glob("source_images\\%s\\%s\\*" %(part, current_session))[-1] #get path for last image in sequence, which contains the emotion
            #print(sourcefile_emotion)
            sourcefile_neutral = glob.glob("source_images\\%s\\%s\\*" %(part, current_session))[0] #do same for neutral image
            
            dest_neut = "sorted_set\\neutral\\%s" %sourcefile_neutral[25:] #Generate path to put neutral image
            dest_emot = "sorted_set\\%s\\%s" %(emotions[emotion], sourcefile_emotion[25:]) #Do same for emotion containing image
            
            copyfile(sourcefile_neutral, dest_neut) #Copy file
            copyfile(sourcefile_emotion, dest_emot) #Copy


001
002
004
006
001
002
003
004
005
006
001
002
003
005
001
005
006
001
002
003
006
001
001
001
003
004
005
006
001
003
005
001
005
006
001
003
006
001
002
004
006
001
003
006
004
005
001
002
003
004
001
002
004
006
002
003
001
004
006
001
004
002
003
004
001
003
004
005
006
002
003
004
001
003
006
001
005
006
001
002
002
003
005
001
002
004
001
002
004
005
001
002
001
003
004
002
003
004
005
002
003
004
005
002
004
005
006
002
003
004
005
002
003
004
002
003
005
001
002
004
005
006
005
006
001
006
001
002
004
005
002
005
006
008
001
005
006
001
006
001
004
007
001
002
004
001
005
008
001
002
008
001
005
007
003
001
002
002
003
004
001
002
001
004
005
007
001
004
001
002
003
002
006
007
001
003
001
003
004
001
004
001
004
001
006
007
010
001
003
004
001
004
006
003
004
001
004
007
002
005
006
002
002
003
009
008
002
004
006
001
005
005
006
008
003
005
006
001
001
006
007
005
001
003
008
001
006
001
004
008
001
006
007
001
003
006
001
003
008
001
001
003
006
007
001
005
006
007
008
004


In [27]:
#makesets.py

import cv2
import glob

faceDet = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
faceDet_two = cv2.CascadeClassifier("haarcascade_frontalface_alt2.xml")
faceDet_three = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")
faceDet_four = cv2.CascadeClassifier("haarcascade_frontalface_alt_tree.xml")

emotions = ["neutral", "anger", "contempt", "disgust", "fear", "happy", "sadness", "surprise"] #Define emotions

In [39]:
def detect_faces(emotion):
    files = glob.glob("sorted_set\\%s\\*" %emotion) #Get list of all images with emotion
    #print(files)
    filenumber = 0
    for f in files:
        frame = cv2.imread(f, 1) #Open image
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) #Convert image to grayscale
        
        #Detect face using 4 different classifiers
        face = faceDet.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)
        
        face_two = faceDet_two.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)
        
        face_three = faceDet_three.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)
       
        face_four = faceDet_four.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)
        

        #Go over detected faces, stop at first detected face, return empty if no face.
        if len(face) == 1:
            #print(face)
            #print(len(face))
            facefeatures = face
        elif len(face_two) == 1:
            #print(face_two)
            facefeatures = face_two
        elif len(face_three) == 1:
            #print(face_three)
            facefeatures = face_three
        elif len(face_four) == 1:
            #print(face_four)
            facefeatures = face_four
        else:
            facefeatures = ""
        
        #Cut and save face
        for (x, y, w, h) in facefeatures: #get coordinates and size of rectangle containing face
            #print ("face found in file: %s" %f)
            gray = gray[y:y+h, x:x+w] #Cut the frame to size
            
            try:
                out = cv2.resize(gray, (350, 350)) #Resize face so all images have same size
                cv2.imwrite("dataset\\%s\\%s.jpg" %(emotion, filenumber), out) #Write image
                filenumber +=1
                #print(filenumber)
            except:
               pass #If error, pass file
               filenumber += 1 #Increment image number
    print('Finish detect_faces')

for emotion in emotions:
    print('hi\n')
    detect_faces('surprise') #Call functiona

error: OpenCV(3.4.1) C:\bld\opencv_1520732670222\work\opencv-3.4.1\modules\imgproc\src\color.cpp:11147: error: (-215) scn == 3 || scn == 4 in function cv::cvtColor


In [93]:
import cv2
import glob
import random
import numpy as np

emotions = ["neutral", "anger", "contempt", "disgust", "fear", "happy", "sadness", "surprise"] #Emotion list
fishface = cv2.face.FisherFaceRecognizer_create() #Initialize fisher face classifier

data = {}

def get_files(emotion): #Define function to get file list, randomly shuffle it and split 80/20
    files = glob.glob("dataset\\%s\\*" %emotion)
    random.shuffle(files)
    training = files[:int(len(files)*0.8)] #get first 80% of file list
    prediction = files[-int(len(files)*0.2):] #get last 20% of file list
    return training, prediction

def make_sets():
    training_data = []
    training_labels = []
    prediction_data = []
    prediction_labels = []
    for i in range(0,7):
        emotion=emotions[i]
        print(emotion)
        training, prediction = get_files(emotion)
        #Append data to training and prediction list, and generate labels 0-7
        for item in training:
            image = cv2.imread(item) #open image
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) #convert to grayscale
            training_data.append(gray) #append image array to training data list
            training_labels.append(emotions.index(emotion))
    
        for item in prediction: #repeat above process for prediction set
            image = cv2.imread(item)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            print('lol')
            prediction_data.append(gray)
            print(prediction_data)
            prediction_labels.append(emotions.index(emotion))
        

    return training_data, training_labels, prediction_data, prediction_labels


def run_recognizer():
    training_data, training_labels, prediction_data, prediction_labels = make_sets()
   
    
    print("training fisher face classifier")
    print("size of training lables is:", len(np.asarray(training_labels)), "images")
    print("size of training data is:", len(training_data), "images")
    fishface.train(training_data, np.asarray(training_labels))

    print("predicting classification set")
    cnt = 0
    correct = 0
    incorrect = 0
    for image in prediction_data:
        pred, conf = fishface.predict(image)
        if pred == prediction_labels[cnt]:
            correct += 1
            cnt += 1
        else:
            incorrect += 1
            cnt += 1
    return ((100*correct)/(correct + incorrect))

metascore = []
for i in range(0,10):
    correct = run_recognizer()
    print("got", correct, "percent correct!")
    metascore.append(correct)

print("\n\nend score:", np.mean(metascore), "percent correct!")


neutral


error: OpenCV(3.4.1) C:\bld\opencv_1520732670222\work\opencv-3.4.1\modules\imgproc\src\color.cpp:11147: error: (-215) scn == 3 || scn == 4 in function cv::cvtColor


In [ ]:
def run_recognizer():
    training_data, training_labels, prediction_data, prediction_labels = make_sets()
   
    
    print("training fisher face classifier")
    print("size of training lables is:", len(np.asarray(training_labels)), "images")
    print("size of training data is:", len(training_data), "images")
    fishface.train(training_data, np.asarray(training_labels))

    print("predicting classification set")
    cnt = 0
    correct = 0
    incorrect = 0
    for image in prediction_data:
        pred, conf = fishface.predict(image)
        if pred == prediction_labels[cnt]:
            correct += 1
            cnt += 1
        else:
            incorrect += 1
            cnt += 1
    return ((100*correct)/(correct + incorrect))

In [ ]:
metascore = []
for i in range(0,10):
    correct = run_recognizer()
    print("got", correct, "percent correct!")
    metascore.append(correct)

print("\n\nend score:", np.mean(metascore), "percent correct!")

In [15]:
import os
facefeature="";
camera_port = 0
ramp_frames = 30
camera = cv2.VideoCapture(camera_port)
def get_image():
    
    retval, im = camera.read()
    return im

temp = get_image()
print("Taking Image")
camera_capture = get_image()
#try:
#    os.remove("capture/capture_image.png")
#    os.remove("capture/capture1.jpg")

#except:
#    print("No files")
file = "capture/capture_image.png"
cv2.imwrite(file, camera_capture)

del(camera)

Taking Image


In [20]:
def detect_faces():
    global facefeature
    files = glob.glob("capture/*") 
    filenumber = 0
    #print(files)
    for f in files:
        print(f)
        frame = cv2.imread(f, 0) 
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        face = faceDet.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)
        face2 = faceDet_two.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)
        face3 = faceDet_three.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)
        face4 = faceDet_four.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)
        if len(face) == 1:
            facefeature = face
        elif len(face2) == 1:
            facefeature == face2
        elif len(face3) == 1:
            facefeature = face3
        elif len(face4) == 1:
            facefeature = face4
        else:
            facefeature = ""
        print(facefeature)
        for (x, y, w, h) in facefeature: 
            print ("face found in file: %s" %f)
            gray = gray[y:y+h, x:x+w] 
            try:
                out = cv2.resize(gray, (350, 350)) 
                cv2.imwrite("capture/capture1.jpg" , out)
                print("Face cropped and saved\n")
            except:
                pass 
                filenumber += 1
detect_faces()

capture\capture_image.png


error: OpenCV(3.4.1) C:\bld\opencv_1520732670222\work\opencv-3.4.1\modules\imgproc\src\color.cpp:11147: error: (-215) scn == 3 || scn == 4 in function cv::cvtColor


In [181]:
emotions = [ "neutral", "anger", "contempt", "disgust", "fear", "happy", "sadness", "surprise"] 
fishface = cv2.face.FisherFaceRecognizer_create() 

data = {}
def get_files(emotion): 
    print("getting files from emotions %s" %emotion);
    files = glob.glob("dataset/%s/*" %emotion)
    random.shuffle(files)
    training = files[:int(len(files))]
    #prediction = cv2.imread("capture/capture1.jpg")
    return training


def make_sets():
    training_data = []
    training_labels = []
    for emotion in emotions:
        training = get_files(emotion)
        for item in training:
            image = cv2.imread(item) 
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
            training_data.append(gray) 
            training_labels.append(emotions.index(emotion))
            #prediction_labels.append(emotions.index(emotion))
    return training_data, training_labels

def fishfacetrain():
    training_data, training_labels = make_sets()
    print ("training fisher face classifier")
    print ("size of training set is:", len(training_labels), "images")
    if(os.path.isfile("fishface.xml")):
        fishface.read("fishface.xml")
        return
    fishface.train(training_data, np.asarray(training_labels))
    fishface.save("fishface.xml")



def run_recognizer():
    image = cv2.imread("capture/capture1.jpg", 1)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    cv2.imshow('src',gray) 
    cv2.waitKey(1000)
    cv2.destroyAllWindows()  
    fishfacetrain();
    print ("predicting classification set")
    cnt = 0
    correct = 0
    incorrect = 0     
    pred, conf = fishface.predict(gray)
    print("pred = %s" %(emotions[pred]))
    return;
run_recognizer();

getting files from emotions neutral
getting files from emotions anger
getting files from emotions contempt
getting files from emotions disgust
getting files from emotions fear
getting files from emotions happy
getting files from emotions sadness
getting files from emotions surprise
training fisher face classifier
size of training set is: 360 images


error: OpenCV(3.4.1) C:\bld\opencv_1520732670222\work\opencv-3.4.1\modules\core\src\persistence_c.cpp:753: error: (-2) The node is neither a map nor an empty collection in function cvGetFileNodeByName
